# **pyCoreRelator** [![GitHub](https://img.shields.io/badge/GitHub-pyCoreRelator-blue?logo=github)](https://github.com/GeoLarryLai/pyCoreRelator) [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.xxxxxxxx.svg)](https://doi.org/10.5281/zenodo.xxxxxxxx)
## **Workshop Notebook #1: Processing CT Scan Images to Digital Logs**   [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/GeoLarryLai/pyCoreRelator/blob/main/pyCoreRelator_2_CTimg2log.ipynb)
This notebook demonstrates the general workflow for using to modules from **pyCoreRelator** to convert CT scan DICOM files into continuous digital CT number logs.

### Key Functions from **pyCoreRelator**
- **`ct_process_and_stitch()`**: Main orchestration function for complete processing workflow
- **`plot_ctimg_curves()`**: Visualization function creating 3-panel diagnostic plots

For advanced usage, see [FUNCTION_DOCUMENTATION.md](https://github.com/GeoLarryLai/pyCoreRelator/blob/main/FUNCTION_DOCUMENTATION.md) for more details.
<hr>

# **Import Packages**
Load CT image processing functions from **pyCoreRelator**

In [ ]:
from pyCoreRelator import plot_ctimg_curves, ct_process_and_stitch

%matplotlib inline

<hr>

# **Define Core CT Data Structure**

Configure your core segments and processing parameters below. The `ct_data_reading_structure` dictionary defines:

### Data Reading Structure Format
Image segments are processed in dictionary order and stitched from top to bottom.

Each segment requires:
- **`scans`**: List of scan folder names (e.g., `['SE000000', 'SE000002']`)
- **`params`**: Dictionary of processing parameters for each scan:
  - `trim_top`: Pixels to remove from top
  - `trim_bottom`: Pixels to remove from bottom
  - `min_brightness`: Threshold for masking low-intensity regions (typically 400)
  - `buffer`: Buffer zone in mm around masked regions
- **`rgb_pxlength`**: Target pixel length for rescaling to match RGB images (from [pyCoreRelator_1_RGBimg2log.ipynb](https://github.com/GeoLarryLai/pyCoreRelator/blob/main/pyCoreRelator_1_RGBimg2log.ipynb))
- **`rgb_pxwidth`**: Target pixel width for rescaling to match RGB images (from [pyCoreRelator_1_RGBimg2log.ipynb](https://github.com/GeoLarryLai/pyCoreRelator/blob/main/pyCoreRelator_1_RGBimg2log.ipynb))
- **`upside_down`**: Set to `True` to rotate segment 180° (for inverted cores)

### Optional Features

For cores split into sub-sections (A, B, C):
- **`suffixes`**: List like `['A', 'B', 'C']` for multi-section segments
- Parameters use format: `'A/SE000000'`, `'B/SE000000'`, etc.

### Core M9907-23PC

In [ ]:
# # Basic information for the core to be processed
# cruise_name = "M9907"
# core_name = "23PC"
# core_length_cm = 783  # Adjust this value based on actual core length

# # Define critical variables for CT data reading
# ct_data_reading_structure = {
#     f'{cruise_name}-{core_name}-6': {
#         'scans': ['SE000000'], #without cut off the T0
#         'params': {
#             'SE000000': {'trim_top': 62, 'trim_bottom': 532, 'min_brightness': 400, 'buffer': 5}   #to cut off the T0
#         },
#         'rgb_pxlength': 6305, 'rgb_pxwidth': 995,
#         'upside_down': True
#     },
#     f'{cruise_name}-{core_name}-5': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 50, 'trim_bottom': 190, 'min_brightness': 400, 'buffer': 5},
#             'SE000002': {'trim_top': 150, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14965, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-4': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 50, 'trim_bottom': 150, 'min_brightness': 400, 'buffer': 5},
#             'SE000002': {'trim_top': 170, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14945, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-3': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 50, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 20, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 1, 'trim_bottom': 40, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14890, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-2': {
#         'suffixes': ['A', 'B', 'C'],
#         'scans': ['SE000000'],
#         'params': {
#             'A/SE000000': {'trim_top': 45, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'B/SE000000': {'trim_top': 1, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'C/SE000000': {'trim_top': 20, 'trim_bottom': 70, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 11185, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-1': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 65, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 5},
#             'SE000002': {'trim_top': 20, 'trim_bottom': 45, 'min_brightness': 400, 'buffer': 5}
#         },
#         'rgb_pxlength': 14915, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }
# }


### Core M9907-25PC

In [ ]:
# # Basic information for the core to be processed
# cruise_name = "M9907"
# core_name = "25PC"
# core_length_cm = 797

# # Define critical variables for CT data reading
# ct_data_reading_structure = {
#     f'{cruise_name}-{core_name}-6': {
#         'scans': ['SE000002'], 
#         'params': {
#             'SE000002': {'trim_top': 122, 'trim_bottom': 42, 'min_brightness': 400, 'buffer': 3}  
#         },
#         'rgb_pxlength': 7635, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-5': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 60, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 120, 'trim_bottom': 60, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 14958, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-4': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 40, 'trim_bottom': 180, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 20, 'trim_bottom': 45, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 14955, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-3': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 50, 'trim_bottom': 1, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 320, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 14860, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-2': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 80, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 3},
#             'SE000002': {'trim_top': 230, 'trim_bottom': 40, 'min_brightness': 400, 'buffer': 3}
#         },
#         'rgb_pxlength': 11510, 'rgb_pxwidth': 995,
#         'upside_down': False
#     },
#     f'{cruise_name}-{core_name}-1': {
#         'scans': ['SE000000', 'SE000002'],
#         'params': {
#             'SE000000': {'trim_top': 60, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 1},
#             'SE000002': {'trim_top': 130, 'trim_bottom': 5, 'min_brightness': 400, 'buffer': 1}
#         },
#         'rgb_pxlength': 14905, 'rgb_pxwidth': 995,
#         'upside_down': False
#     }
# }


### Core M9907-11PC

In [ ]:
# Basic information for the core to be processed
cruise_name = "M9907"
core_name = "11PC"
core_length_cm = 439

# Define critical variables for CT data reading
ct_data_reading_structure = {
    f'{cruise_name}-{core_name}-3': {
        'scans': ['SE000000', 'SE000002'],
        'params': {
            'SE000000': {'trim_top': 53, 'trim_bottom': 120, 'min_brightness': 400, 'buffer': 3},
            'SE000002': {'trim_top': 480, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 3}
        },
        'rgb_pxlength': 13625, 'rgb_pxwidth': 995,
        'upside_down': False
    },
    f'{cruise_name}-{core_name}-2': {
        'scans': ['SE000000', 'SE000002'],
        'params': {
            'SE000000': {'trim_top': 50, 'trim_bottom': 3, 'min_brightness': 400, 'buffer': 3},
            'SE000002': {'trim_top': 360, 'trim_bottom': 50, 'min_brightness': 400, 'buffer': 3}
        },
        'rgb_pxlength': 15055, 'rgb_pxwidth': 995,
        'upside_down': False
    },
    f'{cruise_name}-{core_name}-1': {
        'scans': ['SE000000', 'SE000002'],
        'params': {
            'SE000000': {'trim_top': 40, 'trim_bottom': 20, 'min_brightness': 400, 'buffer': 1},
            'SE000002': {'trim_top': 270, 'trim_bottom': 47, 'min_brightness': 400, 'buffer': 1}
        },
        'rgb_pxlength': 14885, 'rgb_pxwidth': 995,
        'upside_down': False
    }
}


<hr>

# **Execute the functions:**

## Stitch CT images and convert to digital log data

**Function: `ct_process_and_stitch()`**

**What it does:**
1. Loads and processes DICOM files from multiple scan segments with controlled trimming and masking
2. Automatically finds optimal overlaps between scans, and stitches them into one complete profile
3. Rescale the results to match the RGB image length dimensions and the true core length
7. Exports results to image and csv log files

**Key Parameters:**
- `data_reading_structure` *(dict or list)*: Core structure definition with segment configurations
- `ct_data_dir` *(str)*: Base directory path containing CT scan subdirectories
- `width_start_pct` *(float, default=0.15)*: Starting percentage of width for brightness analysis (crops left side)
- `width_end_pct` *(float, default=0.85)*: Ending percentage of width for brightness analysis (crops right side)
- `max_value_side_trim` *(float, default=1300)*: Pixel intensity threshold for automatic side trimming
- `min_overlap` *(int, default=20)*: Minimum overlap length in pixels for stitching
- `max_overlap` *(int, default=400)*: Maximum overlap length in pixels for stitching
- `vmin` *(float, default=None)*: Minimum value for colormap scaling in visualizations
- `vmax` *(float, default=None)*: Maximum value for colormap scaling in visualizations
- `save_csv` *(bool, default=True)*: Whether to export results to CSV file
- `output_csv` *(str, default=None)*: Full path for output CSV file (required if save_csv=True)
- `total_length_cm` *(float, default=None)*: Total core length in cm for depth conversion (required if save_csv=True)

In [ ]:
final_stitched_slice, final_stitched_brightness, final_stitched_stddev, final_stitched_depth, px_spacing_x, px_spacing_y = ct_process_and_stitch(
    data_reading_structure = ct_data_reading_structure,
    ct_data_dir = "example_data/raw_data/CT_data",
    save_csv = True,
    output_csv = f"example_data/processed_data/{cruise_name}-{core_name}/{cruise_name}-{core_name}_CT.csv",
    vmin = 400,
    vmax = 2400,
    total_length_cm = core_length_cm
)

## Visualize the final stitched results

**Function: `plot_ctimg_curves()`**

**What it does:**
1. Creates a 3-panel figure with:
   - **Left panel:** CT image with jet colormap
   - **Middle panel:** Brightness trace with ±1σ shading
   - **Right panel:** Standard deviation profile
2. Automatically adjusts figure dimensions based on data aspect ratio
3. Saves composite figures in specified formats (png, jpg, svg, pdf)
4. Creates separate compressed TIFF image if 'tiff' format is requested

**Key Parameters:**
- `slice_data` *(numpy.ndarray)*: 2D CT slice data to display
- `brightness` *(numpy.ndarray)*: 1D array of mean brightness values along depth
- `stddev` *(numpy.ndarray)*: 1D array of standard deviation values along depth
- `pixel_spacing` *(tuple, default=None)*: Tuple of (x, y) pixel spacing in mm/pixel for physical scaling
- `core_name` *(str, default="")*: Name of the core for title and filenames
- `save_figs` *(bool, default=False)*: Whether to save figures to files
- `output_dir` *(str, default=None)*: Directory to save figures (required if save_figs=True)
- `vmin` *(float, default=None)*: Minimum value for colormap scaling (defaults to 400 if None)
- `vmax` *(float, default=None)*: Maximum value for colormap scaling (defaults to 2400 if None)
- `fig_format` *(list, default=['png'])*: List of file formats to save (options: 'png', 'jpg', 'svg', 'pdf', 'tiff')
- `dpi` *(int or None, default=None)*: Resolution in dots per inch for saved figures (None uses matplotlib default)

In [ ]:
plot_ctimg_curves(final_stitched_slice,                          
                    final_stitched_brightness,                   
                    final_stitched_stddev,                       
                    pixel_spacing=(px_spacing_x, px_spacing_y),  
                    core_name=f"{cruise_name}-{core_name}_CT",   
                    save_figs = True,                            
                    output_dir = f"example_data/processed_data/{cruise_name}-{core_name}"
)  